In [1]:
# 원-핫 인코딩 적용
import pandas as pd
from sklearn import preprocessing

class2 = pd.read_csv('../data/class2.csv')
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
print(train_x)

array([2, 2, 1, 0, 1, 0])

In [4]:
# 코퍼스에 카운터 벡터 적용
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance.',
]
vect = CountVectorizer()
vect.fit(corpus)
# 학습한 단어 인덱스를 출력
print(vect.vocabulary_)

{'this': 13, 'is': 7, 'last': 8, 'chance': 2, 'and': 0, 'if': 6, 'you': 15, 'do': 3, 'not': 10, 'have': 5, 'will': 14, 'never': 9, 'get': 4, 'any': 1, 'one': 11, 'please': 12}


In [7]:
# 배열 변환
# 주어진 문장의 단어의 개수를 카운팅한 뒤, 사전에서 단어를 찾고, 있으면 그 인덱스에 개수를 저장
vect.transform(['you will never get any chance.']).toarray()

array([[0, 1, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [8]:
# 불용어를 제거한 카운터 벡터
# stop_words를 사용하여 is, not, an 같은 불용어를 제거함
vect = CountVectorizer(stop_words=['and', 'is', 'please', 'this']).fit(corpus)
print(vect.vocabulary_)

{'last': 6, 'chance': 1, 'if': 5, 'you': 11, 'do': 2, 'not': 8, 'have': 4, 'will': 10, 'never': 7, 'get': 3, 'any': 0, 'one': 9}


In [10]:
# TF-IDF를 적용한 후, 행렬로 표현
from sklearn.feature_extraction.text import TfidfVectorizer

doc = ['I like machine learning', 'I love deep learning', 'I run everyday']
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)
print('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), '행렬을 만들었습니다.')
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


In [23]:
# 데이터셋을 메모리로 로딩하고 토큰화 적용
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import gensim
from gensim.models import Word2Vec

sample = open('../data/peter.txt', 'r', encoding='UTF8')
s = sample.read()
# 줄 바꿈을 공백으로 변환
f = s.replace('\n', ' ')
data = []

# 로딩한 파일의 각 문장마다 반복
for i in sent_tokenize(f):
    temp = []
    # 문장을 단어로 토큰화
    for j in word_tokenize(i):
        # 토큰화된 단어를 소문자로 변환하여 temp에 저장
        temp.append(j.lower())
    data.append(temp)

print(data)

[['once', 'upon', 'a', 'time', 'in', 'london', ',', 'the', 'darlings', 'went', 'out', 'to', 'a', 'dinner', 'party', 'leaving', 'their', 'three', 'children', 'wendy', ',', 'jhon', ',', 'and', 'michael', 'at', 'home', '.'], ['after', 'wendy', 'had', 'tucked', 'her', 'younger', 'brothers', 'jhon', 'and', 'michael', 'to', 'bed', ',', 'she', 'went', 'to', 'read', 'a', 'book', '.'], ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'], ['he', 'was', 'flying', '.'], ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'], ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'], ['“', 'hello', '!'], ['who', 'are', 'you', '?'], ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'], ['“', 'my', 'name', 'is', 'peter', 'pan', '.'], ['my', 'shadow', 'wouldn', '’', 't', 'stock', 'to', 'me.', '”', ',', 'he', 'replied', '.'], ['she', 'asked', 'him', 'to', 'come', 'in', '.'], ['peter', 'agreed', 'and', 'came', 'inside', 'the', 'room', '

In [26]:
# 데이터셋에 CBOW 적용 후 'peter'와 'wendy'의 유사성 확인
model1 = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5, sg=0)
# 결과 출력
print("Cosine similarity between 'peter' " + "'wendy' - CBOW : ", model1.wv.similarity('peter', 'wendy'))

Cosine similarity between 'peter' 'wendy' - CBOW :  0.074393824


In [28]:
# 'peter'와 'hook'의 유사성 확인
print("Cosine similarity between 'peter' " + "'hook' - CBOW : ", model1.wv.similarity('peter', 'hook'))

Cosine similarity between 'peter' 'hook' - CBOW :  0.027709836


In [29]:
# 데이터셋에 skip-gram 적용 후 'peter'와 'wendy'의 유사성 확인
# skip-gram 모델 사용
model2 = gensim.models.Word2Vec(data, min_count=1, vector_size=100, window=5, sg=1)
# 결과 출력
print("Cosine similarity between 'peter' " + "'wendy' - Skip Gram : ", model2.wv.similarity('peter', 'wendy'))

Cosine similarity between 'peter' 'wendy' - Skip Gram :  0.40088683


In [32]:
# 'peter'와 'hook'의 유사성
# peter와 hook이 의미적으로 얼마나 가까운지 평가하는 것, 유사성을 판단
print("Cosine similarity between 'peter' " + "'hook' - Skip Gram : ", model2.wv.similarity('peter', 'hook'))

Cosine similarity between 'peter' 'hook' - Skip Gram :  0.5201673


In [33]:
# 라이브러리 및 데이터 호출
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText('../data/peter.txt', vector_size=4, window=3, min_count=1, epochs=10)

In [34]:
# 'peter', 'wendy'에 대한 코사인 유사도
sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

0.4592452


In [35]:
# 'peter'와 'hook'에 대한 코사인 유사도
sim_score = model.wv.similarity('peter', 'hook')
print(sim_score)

0.043825686


In [36]:
# 라이브러리와 사전 훈련된 모델 호출
from __future__ import print_function
# gensim은 자연어를 벡터로 변환하는 데 필요한 편의 기능을 제공하는 라이브러리
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format('../data/wiki.ko.vec')

In [57]:
# '도경'과 유사한 단어와 유사도 확인
find_similar_to = '도경'

for similar_word in model_kr.similar_by_word(find_similar_to):
    print(f'Word: {similar_word[0]}, Similar: {similar_word[1] * 100}%')

# dir(model_kr)
# model_kr.key_to_index

Word: 박도경, Similar: 83.59906673431396%
Word: 이도경, Similar: 82.49361515045166%
Word: 권도경, Similar: 80.06789088249207%
Word: 도경수, Similar: 79.98784184455872%
Word: 도경리, Similar: 78.66216897964478%
Word: 하경, Similar: 78.43520641326904%
Word: 최호경, Similar: 77.53516435623169%
Word: 송욱경, Similar: 77.4889349937439%
Word: 강문경, Similar: 77.4862289428711%
Word: 옥도경, Similar: 77.2731363773346%


In [59]:
# '동물', '육식동물'에는 긍정적이지만, '사람'에는 부정적인 단어와 유사도 확인
similarities = model_kr.most_similar(positive=['동물', '육식동물'], negative=['사람'])
print(similarities)

[('초식동물', 0.7804122567176819), ('거대동물', 0.7547270655632019), ('육식동물의', 0.7547166347503662), ('유두동물', 0.753511369228363), ('반추동물', 0.7470757961273193), ('독동물', 0.7466291785240173), ('육상동물', 0.746031641960144), ('유즐동물', 0.7450903654098511), ('극피동물', 0.7449344396591187), ('복모동물', 0.742434561252594)]


In [68]:
# 라이브러리 호출 및 데이터셋 로딩
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('C:/projects/pytorch/080289/chap10/data/glove.6B.100d.txt')
# 글로브 데이터를 워드투벡터 형태로 변환
word2vec_glove_file = get_tmpfile('glove.6B.100d.word2vec.txt')
print(glove2word2vec(glove_file, word2vec_glove_file))

(400000, 100)

In [70]:
# 'bill'과 유사한 단어의 리스트를 반환
# load_word2_vec_format() 메서드를 이용하여 word2vec.c 형식으로 벡터를 가져옵니다.

model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
# 단어(bill)을 기준으로 가장 유사한 단어의 리스트를 보여 줍니다.
print(model.most_similar('bill'))

[('legislation', 0.8072139620780945), ('proposal', 0.730686366558075), ('senate', 0.7142541408538818), ('bills', 0.704440176486969), ('measure', 0.6958035230636597), ('passed', 0.690624475479126), ('amendment', 0.6846879720687866), ('provision', 0.6845567226409912), ('plan', 0.6816462874412537), ('clinton', 0.6663140058517456)]


In [71]:
# 'cherry'와 유사한 단어의 리스트를 반환
# 단어(cherry) 기준으로 가장 유사한 단어의 리스트를 보여 줌
model.most_similar('cherry')

[('peach', 0.688809871673584),
 ('mango', 0.683819055557251),
 ('plum', 0.6684104204177856),
 ('berry', 0.6590359210968018),
 ('grove', 0.658155083656311),
 ('blossom', 0.6503506302833557),
 ('raspberry', 0.6477391719818115),
 ('strawberry', 0.6442098021507263),
 ('pine', 0.6390928626060486),
 ('almond', 0.6379212737083435)]

In [72]:
# 'cherry'와 관련성이 없는 단어의 리스트를 반환
# 단어와 관련성이 없는 단어를 반환
model.most_similar(negative=['cherry'])

[('kazushige', 0.48343509435653687),
 ('askerov', 0.4778185784816742),
 ('lakpa', 0.46915265917778015),
 ('ex-gay', 0.4571332633495331),
 ('tadayoshi', 0.4522107243537903),
 ('turani', 0.44810065627098083),
 ('saglam', 0.4469599425792694),
 ('aijun', 0.4435270130634308),
 ('adjustors', 0.44235292077064514),
 ('nyum', 0.4423118233680725)]

In [79]:
# 'woman', 'king'과 유사성이 높으면서 'man'과 관련성이 없는 단어를 반환
result = model.most_similar(positive=['woman', 'king'], negative=['man'])
print(f'{result[0][0]}: {result[0][1]:.4f}')

queen: 0.7699


In [80]:
# 'australia' 'beer', 'france'와 관련성이 있는 단어를 변환
def analogy(x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]
analogy('australia', 'beer', 'france')

'champagne'

In [82]:
# 'tail', 'tallest', 'long' 단어를 기반으로 새로운 단어를 유추
print(analogy('tall', 'tallest', 'long'))

longest


In [84]:
# 'breakfast cereal dinner lunch' 중 유사도가 낮은 단어를 반환
# 유사도가 가장 낮은 단어를 반환
print(model.doesnt_match('breakfast cereal dinner lunch'.split()))

cereal
